# Initial Data Cleaning and Stats Exploration

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cancer-issue/cancer issue.csv


In [2]:
#Import graphing libraries
import seaborn as sns
import matplotlib.pyplot as plt 

In [3]:
#Create first DataFrame
data = pd.read_csv('/kaggle/input/cancer-issue/cancer issue.csv')

#Basic Info on variable types, possible nulls for investigation
#data.info() #--17,686 records total
#data.isna().sum() #--4,326 NaN records in GeneticMarker field
cl_data = data.dropna(axis=0, how='any') #-- cleaned data with genetic marker NaN cleaned out
print(cl_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 13360 entries, 1 to 17684
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PatientID          13360 non-null  int64  
 1   Age                13360 non-null  int64  
 2   Gender             13360 non-null  object 
 3   Race/Ethnicity     13360 non-null  object 
 4   BMI                13360 non-null  float64
 5   SmokingStatus      13360 non-null  object 
 6   FamilyHistory      13360 non-null  object 
 7   CancerType         13360 non-null  object 
 8   Stage              13360 non-null  object 
 9   TumorSize          13360 non-null  float64
 10  TreatmentType      13360 non-null  object 
 11  TreatmentResponse  13360 non-null  object 
 12  SurvivalMonths     13360 non-null  int64  
 13  Recurrence         13360 non-null  object 
 14  GeneticMarker      13360 non-null  object 
 15  HospitalRegion     13360 non-null  object 
dtypes: float64(2), int64(3), ob

In [4]:
#Stat descriptions for numeric fields after data cleaning
cl_data.describe()

,PatientID,Age,BMI,TumorSize,SurvivalMonths
count,13360.000000,13360.000000,13360.000000,13360.000000,13360.000000
mean,8839.728518,53.796632,29.226684,5.484581,60.487350
std,5098.236171,21.092605,6.212583,2.599659,34.829083
min,2.000000,18.000000,18.500000,1.000000,1.000000
25%,4421.750000,35.000000,23.900000,3.200000,30.000000
50%,8867.000000,54.000000,29.200000,5.500000,60.000000
75%,13231.250000,72.000000,34.600000,7.700000,91.000000
max,17685.000000,90.000000,40.000000,10.000000,120.000000


In [5]:
print(cl_data['CancerType'].value_counts())

CancerType
Leukemia    2269
Skin        2264
Breast      2236
Prostate    2217
Lung        2189
Colon       2185
Name: count, dtype: int64


# Gender Vs. Cancer Type:
* First analysis focuses on exploration if gender is a significant factor in cancer data overall or within analysis of types
* Conclusion: None of the cancer types appear to group significantly by gender within this dataset (before or after cleaning)

In [6]:
#First, analyze the raw data (prior to dropna for Genetic Marker)
data_gender_split = data['Gender'].value_counts()
print(data_gender_split)

'''
Results:
Female    8930
Male      8756
Spread of 174 records
'''

#Second, analyze cl_data
cl_data_gender_split = cl_data['Gender'].value_counts()
print(cl_data_gender_split)
'''
Results:
Gender
Female    6743
Male      6617
Spread of 126 records
'''

Gender
Female    8930
Male      8756
Name: count, dtype: int64
Gender
Female    6743
Male      6617
Name: count, dtype: int64


'\nResults:\nGender\nFemale    6743\nMale      6617\nSpread of 126 records\n'

In [7]:
#Analysis of gender vs. cancer type displayed numerically
gender_vs_type = pd.crosstab(cl_data['Gender'], cl_data['CancerType'], margins = True)
gender_vs_type

CancerType,Breast,Colon,Leukemia,Lung,Prostate,Skin,All
Gender,,,,,,,
Female,1079,1101,1144,1133,1096,1190,6743
Male,1157,1084,1125,1056,1121,1074,6617
All,2236,2185,2269,2189,2217,2264,13360


In [8]:
#Analysis of gender vs. cancer type displayed in %'s'
gender_vs_type_perc = pd.crosstab(cl_data['Gender'], cl_data['CancerType'], normalize='columns', margins = True)*100
gender_vs_type_perc = gender_vs_type_perc.round(2).astype(str) + '%'
gender_vs_type_perc

CancerType,Breast,Colon,Leukemia,Lung,Prostate,Skin,All
Gender,,,,,,,
Female,48.26%,50.39%,50.42%,51.76%,49.44%,52.56%,50.47%
Male,51.74%,49.61%,49.58%,48.24%,50.56%,47.44%,49.53%


# Smoking Vs. Cancer Type:
* Next we can check if smoking status shows any effect on cancer type
* Smokers and Past Smokers share roughly 1/3 of the entries each (2/3 total) vs. Non Smokers
* Data for Smokers and Past Smokers was combined to show overall % effect from smoking
* Conclusion: Smoking (past or current) is significantly represented vs. Non Smokers in this dataset

In [9]:
#Analysis of smoking status vs. cancer type
smoke_vs_type = pd.crosstab(cl_data['SmokingStatus'], cl_data['CancerType'], margins = True)
smoke_vs_type

CancerType,Breast,Colon,Leukemia,Lung,Prostate,Skin,All
SmokingStatus,,,,,,,
Former Smoker,751,763,779,713,725,764,4495
Non-Smoker,771,723,766,740,748,742,4490
Smoker,714,699,724,736,744,758,4375
All,2236,2185,2269,2189,2217,2264,13360


In [10]:
#Since smoking and former smoking seem to represent a larger margin, we can quickly combine the categories with a lambda function
#Note: I had to explicitly create cl_data as a copy here since the lambda function was generating a SettingWithCopyWarning otherwise
cl_data = cl_data.copy()
cl_data['Smoke_Comp'] = cl_data.apply(lambda x: 'Smoker' if x['SmokingStatus'] in ['Former Smoker', 'Smoker'] else 'Non-Smoker', axis=1)
smoke_vs_type_fix = pd.crosstab(cl_data['Smoke_Comp'], cl_data['CancerType'], margins = True)
smoke_vs_type_fix

CancerType,Breast,Colon,Leukemia,Lung,Prostate,Skin,All
Smoke_Comp,,,,,,,
Non-Smoker,771,723,766,740,748,742,4490
Smoker,1465,1462,1503,1449,1469,1522,8870
All,2236,2185,2269,2189,2217,2264,13360


In [11]:
#There is a clear bias toward smokers and past smokers for cancer types
smoke_vs_type_fix_perc = pd.crosstab(cl_data['Smoke_Comp'], cl_data['CancerType'], margins = True, normalize='columns')*100
smoke_vs_type_fix_perc = smoke_vs_type_fix_perc.round(0).astype(str)+'%'
smoke_vs_type_fix_perc

CancerType,Breast,Colon,Leukemia,Lung,Prostate,Skin,All
Smoke_Comp,,,,,,,
Non-Smoker,34.0%,33.0%,34.0%,34.0%,34.0%,33.0%,34.0%
Smoker,66.0%,67.0%,66.0%,66.0%,66.0%,67.0%,66.0%
